In [ ]:
import datetime
import time
import xml.etree.ElementTree as ElementTree
from collections import defaultdict
import requests
import pandas as pd
import numpy as np
%matplotlib inline

For the production data, we should go to [this website](https://servicio.mapama.gob.es/InformacionMercado_Aica/index.jsp?aplica=IMA) and check the cookie header so we can use this piece of code for get the data in an automated way.

In [ ]:
# https://servicio.mapama.gob.es/InformacionMercado_Aica/index.jsp?aplica=IMA
# copy cookie
headers = {
    'cache-control': "no-cache",
    'Cookie': 'JSESSIONID=6F6F84C286242686BC8D31FE1D27BA91.e0110050' ## The cookie can be used for a few minutes
}
url = 'https://servicio.mapama.gob.es/InformacionMercado_Aica/Prod_existenciasXML.aao?actividad=1&campa={last_year}{year}&mes_ant={month}'
production_data = []
for year in range(2007, 2019):
    current_production = defaultdict(float)
    for month in range(1, 13):
        month_url = url.format(last_year=year - 1, year=year, month=month)
        for row in ElementTree.fromstring(requests.request("GET", month_url, headers=headers).text):
            cells = list(row)
            provincia = cells[3].text
            current_tons = float(cells[7].text)
            production_data.append({
                'provincia': provincia,
                'production_tons': current_tons - current_production[provincia],
                'year_month': f'{year}-{month:02}'
            })
            current_production[provincia] = current_tons